In [1]:
from data.dataset import NERDataset
from models.networks import GlobalContextualDeepTransition
from tqdm.notebook import tqdm
import torch

In [2]:
sourceName = 'data/conll03/eng.train.src'
targetName = 'data/conll03/eng.train.trg'
data = NERDataset(sourceName, targetName)
loader = data.getLoader(4096)

In [3]:
max(data.wordIdx.values()), len(data.wordIdx)

(23624, 23625)

In [4]:
numChars = 100
charEmbedding = 128
numWords = len(data.wordIdx)
wordEmbedding = 300
contextOutputUnits = 128
contextTransitionNumber = transitionNumber = 4
encoderUnits = 256
decoderUnits = 256

In [5]:
model = GlobalContextualDeepTransition(numChars, charEmbedding, numWords,
                     wordEmbedding, contextOutputUnits, contextTransitionNumber,
                        encoderUnits, decoderUnits, transitionNumber, data.numTags)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {numParams:,}")
# 6,774,656

Trainable parameters: 6,788,497


In [6]:
# x = model.parameters()
# y = model.state_dict().items()

# for (key, value), par in zip(y, x):
#     if par.requires_grad:
#         print(par.numel())

In [7]:
model = model.putOnCuda()
tot = 0
for batch in tqdm(loader):
    words, chars, mask, targets = batch
    batch = words.cuda(), chars.cuda(), mask.cuda(), targets.cuda()
    loss = model.training_step(batch, 0)
    tot += loss.item()

In [8]:
print(f'Total loss is {tot}')

Total loss is 142.85319709777832
